<a href="https://colab.research.google.com/github/widnu/Predictive_Policing_DL/blob/master/nz_dl_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# uploaded = files.upload()

from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd 
#df=pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset/nz_crime_dataset_14151617.csv', encoding='utf-8-sig')*
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset/nz_crime_dataset.csv', encoding='utf-8-sig')

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167347 entries, 0 to 1167346
Data columns (total 17 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   DATE_NO_TIME           1167347 non-null  object 
 1   DATE_TIME              1167347 non-null  object 
 2   7_DAYS_CRIME           1167347 non-null  float64
 3   1_MONTH_CRIME          1167347 non-null  float64
 4   1_YEAR_CRIME           1167347 non-null  float64
 5   TIME_SINCE_LAST_CRIME  1167347 non-null  float64
 6   MONTH                  1167347 non-null  int64  
 7   QUARTER                1167347 non-null  float64
 8   DAY_OF_WEEK            1167347 non-null  object 
 9   DAY                    1167347 non-null  float64
 10  HOUR                   1167347 non-null  float64
 11  HOUR_PARTITION         1167347 non-null  float64
 12  MESHBLOCK              1167347 non-null  int64  
 13  AREA_0                 1167347 non-null  object 
 14  AREA_1            

,7_DAYS_CRIME,1_MONTH_CRIME,1_YEAR_CRIME,TIME_SINCE_LAST_CRIME,MONTH,QUARTER,DAY,HOUR,HOUR_PARTITION,MESHBLOCK,WEAPON_TYPE
count,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06
mean,8.766996e+00,9.575736e+00,1.205730e+01,1.281016e-01,6.536036e+00,2.512512e+00,5.013381e+00,1.310084e+01,1.205440e+01,1.302863e+06,9.731468e-03
std,1.000510e+00,2.638901e-01,5.601060e-01,7.455829e+00,3.552330e+00,1.149184e+00,2.150864e+00,4.183061e+00,4.167649e+00,8.577749e+05,1.042445e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.461000e+03,0.000000e+00
25%,8.377701e+00,9.481893e+00,1.217237e+01,0.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,1.262178e+01,1.200000e+01,6.075000e+05,0.000000e+00
50%,9.071538e+00,9.612667e+00,1.222303e+01,0.000000e+00,7.000000e+00,3.000000e+00,6.000000e+00,1.317855e+01,1.200000e+01,1.167000e+06,0.000000e+00
75%,9.474011e+00,9.706986e+00,1.224577e+01,0.000000e+00,1.000000e+01,4.000000e+00,7.000000e+00,1.400000e+01,1.200000e+01,2.029200e+06,0.000000e+00
max,1.000983e+01,1.023002e+01,1.235463e+01,5.770000e+02,1.200000e+01,4.000000e+00,7.000000e+00,2.300000e+01,2.100000e+01,3.210003e+06,2.000000e+00


In [ ]:
#######################################################
# Count total NaN at each column in DataFrame
print(df.isnull().sum())

# Count all NaN in a DataFrame (both columns & Rows)
print(df.isnull().sum().sum())

# erase every row (axis=0) that has "any" Null value in it.
df = df.dropna(how='any',axis=0) 

DATE_NO_TIME             0
DATE_TIME                0
7_DAYS_CRIME             0
1_MONTH_CRIME            0
1_YEAR_CRIME             0
TIME_SINCE_LAST_CRIME    0
MONTH                    0
QUARTER                  0
DAY_OF_WEEK              0
DAY                      0
HOUR                     0
HOUR_PARTITION           0
MESHBLOCK                0
AREA_0                   0
AREA_1                   0
WEAPON_TYPE              0
CRIME_TYPE               0
dtype: int64
0


In [ ]:
#######################################################
feature_var = ['7_DAYS_CRIME', '1_MONTH_CRIME', '1_YEAR_CRIME', 'TIME_SINCE_LAST_CRIME', 'MONTH', 'DAY', 'QUARTER', 'HOUR_PARTITION', 'AREA_0', 'AREA_1', 'WEAPON_TYPE']
# feature_var = ['AREA_0', 'AREA_1']
response_var = 'CRIME_TYPE'
#######################################################
X = df[feature_var]
y = df.pop(response_var)
# y = df[response_var]

In [ ]:
########################################
# Encode categorical variables
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
oe.fit(X)
X = oe.transform(X)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=5)
# X = pca.fit_transform(X)

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X = sc.fit_transform(X)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# from keras.utils import to_categorical
# y = to_categorical(y)

#######################################################
from imblearn.over_sampling import SMOTE
from collections import Counter
counter = Counter(y)
print(counter)
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

#######################################################
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5)

# Reserve 10,000 samples for validation
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

#######################################################
#df_X = pd.DataFrame(data=X)
#df_y = pd.DataFrame(data=y)

Counter({1: 676408, 2: 371126, 0: 119813})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Counter({1: 676408, 2: 676408, 0: 676408})


In [ ]:
import tensorflow as tf
#dataset = tf.data.Dataset.from_tensor_slices((df_X.values, df_y.values))

In [ ]:
#train_dataset = dataset.shuffle(len(df_X)).batch(1)

In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(3, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model


In [ ]:
model = get_compiled_model()

history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=256,
    validation_data=(X_val, y_val),
)

history.history
model.summary()

# https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

##################################################
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

Epoch 1/256
22039/22039 [==============================] - 52s 2ms/step - loss: 0.9214 - accuracy: 0.5447 - val_loss: 0.9092 - val_accuracy: 0.5478
Epoch 2/256
11842/22039 [===============>..............] - ETA: 22s - loss: 0.9038 - accuracy: 0.5559

KeyboardInterrupt: ignored